## 移除最后conv最后一层网络


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
batch_size = 256
num_epochs = 300
extract_size = 1024
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_conv.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_remove_last_conv.h')

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [4]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = FinalClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
# classify_model.load_state_dict(torch.load('./models/class_single_rm1layer_200.pkl'))

### time to train model

In [9]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [10]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(201, num_epochs+201):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm1layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm1layer_last.pkl'))

Epoch [201/300], Step [120/127], Loss: 1.0929, Accuracy: 71.48%, Best_acc: 78.91%            
 Epoch 201, spent time:6.22s
Epoch [202/300], Step [120/127], Loss: 0.9626, Accuracy: 76.56%, Best_acc: 78.91%            
 Epoch 202, spent time:5.60s
Epoch [203/300], Step [120/127], Loss: 0.9455, Accuracy: 77.34%, Best_acc: 79.69%            
 Epoch 203, spent time:5.68s
Epoch [204/300], Step [120/127], Loss: 1.0117, Accuracy: 73.05%, Best_acc: 79.69%            
 Epoch 204, spent time:5.69s
Epoch [205/300], Step [120/127], Loss: 0.8654, Accuracy: 79.30%, Best_acc: 84.77%            
 Epoch 205, spent time:5.60s
Epoch [206/300], Step [120/127], Loss: 0.9885, Accuracy: 78.12%, Best_acc: 84.77%            
 Epoch 206, spent time:5.54s
Epoch [207/300], Step [120/127], Loss: 1.0404, Accuracy: 74.22%, Best_acc: 84.77%            
 Epoch 207, spent time:5.65s
Epoch [208/300], Step [120/127], Loss: 0.8965, Accuracy: 80.08%, Best_acc: 84.77%            
 Epoch 208, spent time:5.66s
Epoch [209/300],

Epoch [267/300], Step [120/127], Loss: 0.9741, Accuracy: 76.95%, Best_acc: 84.77%            
 Epoch 267, spent time:5.75s
Epoch [268/300], Step [120/127], Loss: 0.9621, Accuracy: 73.83%, Best_acc: 84.77%            
 Epoch 268, spent time:5.66s
Epoch [269/300], Step [120/127], Loss: 0.9193, Accuracy: 75.39%, Best_acc: 84.77%            
 Epoch 269, spent time:5.55s
Epoch [270/300], Step [120/127], Loss: 0.9513, Accuracy: 76.95%, Best_acc: 84.77%            
 Epoch 270, spent time:5.59s
Epoch [271/300], Step [120/127], Loss: 0.9900, Accuracy: 72.27%, Best_acc: 84.77%            
 Epoch 271, spent time:5.66s
Epoch [272/300], Step [120/127], Loss: 0.9773, Accuracy: 75.39%, Best_acc: 84.77%            
 Epoch 272, spent time:5.64s
Epoch [273/300], Step [120/127], Loss: 1.0192, Accuracy: 73.83%, Best_acc: 84.77%            
 Epoch 273, spent time:5.66s
Epoch [274/300], Step [120/127], Loss: 0.9472, Accuracy: 76.95%, Best_acc: 84.77%            
 Epoch 274, spent time:5.76s
Epoch [275/300],

Epoch [333/300], Step [120/127], Loss: 0.9736, Accuracy: 76.17%, Best_acc: 84.77%            
 Epoch 333, spent time:5.63s
Epoch [334/300], Step [120/127], Loss: 0.8927, Accuracy: 78.52%, Best_acc: 84.77%            
 Epoch 334, spent time:5.72s
Epoch [335/300], Step [120/127], Loss: 0.9326, Accuracy: 77.34%, Best_acc: 84.77%            
 Epoch 335, spent time:5.64s
Epoch [336/300], Step [120/127], Loss: 0.9475, Accuracy: 75.00%, Best_acc: 84.77%            
 Epoch 336, spent time:5.62s
Epoch [337/300], Step [120/127], Loss: 1.0003, Accuracy: 74.22%, Best_acc: 84.77%            
 Epoch 337, spent time:5.60s
Epoch [338/300], Step [120/127], Loss: 0.9493, Accuracy: 78.52%, Best_acc: 84.77%            
 Epoch 338, spent time:5.67s
Epoch [339/300], Step [120/127], Loss: 0.9197, Accuracy: 78.52%, Best_acc: 84.77%            
 Epoch 339, spent time:5.65s
Epoch [340/300], Step [120/127], Loss: 0.9832, Accuracy: 75.00%, Best_acc: 84.77%            
 Epoch 340, spent time:5.66s
Epoch [341/300],

Epoch [399/300], Step [120/127], Loss: 0.9074, Accuracy: 80.08%, Best_acc: 84.77%            
 Epoch 399, spent time:5.64s
Epoch [400/300], Step [120/127], Loss: 1.0752, Accuracy: 72.27%, Best_acc: 84.77%            
 Epoch 400, spent time:5.63s
Epoch [401/300], Step [120/127], Loss: 0.9672, Accuracy: 77.73%, Best_acc: 84.77%            
 Epoch 401, spent time:5.69s
Epoch [402/300], Step [120/127], Loss: 1.0385, Accuracy: 76.95%, Best_acc: 84.77%            
 Epoch 402, spent time:5.60s
Epoch [403/300], Step [120/127], Loss: 1.0455, Accuracy: 71.88%, Best_acc: 84.77%            
 Epoch 403, spent time:5.66s
Epoch [404/300], Step [120/127], Loss: 0.9886, Accuracy: 74.22%, Best_acc: 84.77%            
 Epoch 404, spent time:5.70s
Epoch [405/300], Step [120/127], Loss: 1.0679, Accuracy: 72.66%, Best_acc: 84.77%            
 Epoch 405, spent time:5.62s
Epoch [406/300], Step [120/127], Loss: 0.9787, Accuracy: 73.83%, Best_acc: 84.77%            
 Epoch 406, spent time:5.75s
Epoch [407/300],

Epoch [465/300], Step [120/127], Loss: 1.0447, Accuracy: 75.78%, Best_acc: 84.77%            
 Epoch 465, spent time:5.72s
Epoch [466/300], Step [120/127], Loss: 1.0242, Accuracy: 75.00%, Best_acc: 84.77%            
 Epoch 466, spent time:5.69s
Epoch [467/300], Step [120/127], Loss: 1.0470, Accuracy: 72.66%, Best_acc: 84.77%            
 Epoch 467, spent time:5.70s
Epoch [468/300], Step [120/127], Loss: 1.0078, Accuracy: 78.52%, Best_acc: 84.77%            
 Epoch 468, spent time:5.70s
Epoch [469/300], Step [120/127], Loss: 0.9604, Accuracy: 78.12%, Best_acc: 84.77%            
 Epoch 469, spent time:5.73s
Epoch [470/300], Step [120/127], Loss: 0.9847, Accuracy: 74.22%, Best_acc: 84.77%            
 Epoch 470, spent time:5.68s
Epoch [471/300], Step [120/127], Loss: 0.9645, Accuracy: 78.12%, Best_acc: 84.77%            
 Epoch 471, spent time:5.71s
Epoch [472/300], Step [120/127], Loss: 0.9729, Accuracy: 76.95%, Best_acc: 84.77%            
 Epoch 472, spent time:5.72s
Epoch [473/300],

### How good is the model?

In [11]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm1layer.h")

In [12]:
classify_model.load_state_dict(torch.load('./models/class_single_rm1layer_last.pkl'))

In [13]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 70.77%.
